# Blankenbach (et al., 1989) Benchmark

This benchmark is well-suited to test whether a numerical code is capable of modeling mantle convection. Blankenbach et al. (1989) compared various mantle convection models that employed a wide range of numerical methods and produced a list of reference values for steady-state conditions (e.g., Nusselt number or average velocity). As the spatial resolution of our numerical models increases, their results should converge toward these benchmark values.

## Model Setup and Problem Description

Convection is modeled in a rectangular, two-dimensional box with a height of $H$ and a length of $L$ ($H = L = 1000\ \text{km}$). The kinematic boundary conditions are **free-slip** on all sides, and the thermal boundary conditions are **constant temperatures** at the top ($T_{top}$) and bottom ($T_{bottom}$) of the model, and **insulating** $\left(\frac{\partial{T}}{\partial{x}} = 0\right)$ along the lateral boundaries. The temperature difference between the top and bottom is 1000 K for all models. The density in all models depends linearly on temperature and is given by:

$\begin{equation}
\rho = \rho_0 \left(1-\alpha(T-T_{top})\right),
\end{equation}$

where  

- $\rho_0$ is the reference density [kg/m³], and  
- $\alpha$ is the thermal expansion coefficient [1/K].  

Despite the simple configuration, obtaining a steady-state solution is not always straightforward. This is mainly due to (a) the large number of time steps required to reach steady state (often several thousand) and (b) the strong localization of thermal upwellings and downwellings near the model boundaries. This problem becomes particularly severe when the viscosity of the fluid is very low — or, in other words, when convection occurs at a very high **Rayleigh number**.  

The Rayleigh number expresses the ratio of buoyancy forces (which promote convection) to viscous and diffusive forces (which resist it), and is given by:  

$\begin{equation}
Ra = \frac{\rho_0 \alpha \Delta{T} g H^3}{\eta_0 \kappa},
\end{equation}$

where  

- $\Delta{T}$ is the temperature difference [K] between the top and bottom of the model,  
- $g$ is gravitational acceleration [m/s²],  
- $H$ is the thickness of the convecting layer [m],  
- $\eta_0$ is the reference viscosity [Pa·s], and  
- $\kappa=\frac{k}{\rho_0 c_p}$ is the thermal diffusivity [m²/s],  

with  

- $k$ being the thermal conductivity [W/m/K], and  
- $c_p$ the specific heat capacity [J/kg/K].

The reference values for our models are:

$\begin{equation}\begin{split}
g &= 10\ \mathrm{m/s^2}, \\
H &= 1000\ \mathrm{m}, \\
L &= 1000\ \mathrm{m}, \\
T_{top} &= 273\ \mathrm{K}, \\
T_{bottom} &= 1273\ \mathrm{K}, \\
k &= 5\ \mathrm{W/m/K}, \\
c_p &= 1250\ \mathrm{J/kg}, \\
\rho_0 &= 4000\ \mathrm{kg/m³}, \\
\alpha &= 2.5 \times 10^{-5}\ \mathrm{1/K}, \\ 
\eta_0 &= [10^{23},10^{22},10^{21}]\ \mathrm{Pa·s}.
\end{split}\end{equation}$

The localization problem along the sidewalls can be mitigated by increasing the spatial resolution. Ideally, an **irregular grid** would be most efficient (although this has not yet been implemented in `GeoModBox.jl`). Therefore, we must increase the overall grid resolution uniformly.

Depending on the Rayleigh number, we can estimate the required resolution within the **thermal boundary layer** ($d$) at the top (and thus for the entire grid, since a regular grid is used) using:

$\begin{equation}
\left(\frac{H}{d}\right)^3=\frac{1}{4}Ra.
\end{equation}$

Assuming we want to use $n$ grid points within the thermal boundary layer, the total number of grid points can be estimated as:

$\begin{equation}
nz=\left(n-1\right)\sqrt[3]{\frac{Ra}{4}}.
\end{equation}$

## Tasks

Similar to the previous exercises ([11](./11_2D_Thermal_Convection.ipynb) and [12](./12_2D_Thermal_Convection_scaled.ipynb)), we consider purely thermal convection under the **Boussinesq approximation** in its **nondimensional form**. We again use `mutable structures` in Julia instead of named tuples.

Here, we focus solely on **isoviscous thermal convection** for different Rayleigh numbers.

1. Complete the script provided below.  
2. Model three convection cases with reference viscosities of $\eta_0 = 10^{23},\ 10^{22},\ \mathrm{and}\ 10^{21}\ \text{Pa}\cdot{}\text{s}$, and compare the model values (Nusselt number and mean velocity) with the steady-state benchmark values.  
3. For one of the models (e.g., $\eta_0 = 10^{23}$), perform a **resolution test**. (Note: The higher the Rayleigh number, the higher the required resolution! This can become computationally demanding.). Plot the steady-state Nusselt number and mean velocity of the numerical model against the reciprocal grid spacing (1/nx/nz) — ideally using logarithmic axes. The results should approach the benchmark values as the resolution increases.  
4. Summarize and discuss your results in a short report.

Let's first define the necessary modules again:

In [ ]:
using Plots, ExtendableSparse
using GeoModBox
using GeoModBox.InitialCondition, GeoModBox.MomentumEquation.TwoD
using GeoModBox.AdvectionEquation.TwoD, GeoModBox.HeatEquation.TwoD
using GeoModBox.Scaling
using Statistics, LinearAlgebra
using Printf
start=time()

In the following, we define the initial conditions and some parameters for visualization:

> **Note:** The commented scaling factors for the vector arrows correspond to models with different $Ra$ values in ascending order.

In [ ]:
# Define Initial Condition ============================================== #
# Temperature - 
#   1) circle, 2) gaussian, 3) block, 4) linear, 5) lineara
# !!! Gaussian is not working!!!    
Ini         =   (T=:lineara,)
# ----------------------------------------------------------------------- #
# Plot Settings ========================================================= #
Pl  =   (
    qinc        =   10,
    qsc         =   1.0e-3 # 1.0e-3, 4.0e-4, 5.0e-5
)
# Animation Settings ==================================================== #
k           =   scatter()
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
save_fig    =   1
# ----------------------------------------------------------------------- #

In the following, the reference values from the **Blankenbach benchmark** are imported.

> **Note 1:** `Nr` determines which value from the vector is used.  
> The first three values in the benchmark correspond to models with  
> $Ra = 10^4, 10^5,\ \text{and}\ 10^6$.  
> If $Ra$ is changed, `Nr` must be updated accordingly.

> **Note 2:** The last two values correspond to cases with **variable viscosity**.  
> This case still needs to be implemented in this script.

In [ ]:
# Benchmark Values ====================================================== # 
# Taken from Gerya (2019), Introduction to numerical geodynamic modelling
B       =   (
    Nr      =   1,
    # Nusselt Number at the top
    Nu      =   [4.8844,10.534,21.972,10.066,6.9229],   
    # Root Mean Square Velocity-scaled
    Vrms    =   [42.865,193.21,833.99,480.43,171.76],   
    # Non-dimensional temperature gradients in the model corner
    q1      =   [8.0593,19.079,45.964,17.531,18.484],
    q2      =   [0.5888,0.7228,0.8772,1.0085,0.1774],
    q3      =   [8.0593,19.079,45.964,26.809,14.168], 
    q4      =   [0.5888,0.7228,0.8772,0.4974,0.6177],
    # Local minimum along the central vertical temperature profile 
    Tmin    =   [0.4222,0.4284,0.4322,0.7405,0.3970],
    ymin    =   [0.2249,0.1118,0.0577,0.0623,0.1906],
    # Local maximum algon the central vertical temperature profile
    Tmax    =   [0.5778,0.5716,0.5678,0.8323,0.5758],
    ymax    =   [0.7751,0.8882,0.9423,0.8243,0.7837],
)
# ----------------------------------------------------------------------- #

In the following, the geometry and reference parameters of our model are defined.

> **Note:** The default value of $Ra$ in `Physics()` is set to `-9999`,  
> which means that the Rayleigh number must be **explicitly calculated** later in the script.  
> All parameters not defined here will use their **default values**.  
> See the definitions in the corresponding [structures](../../src/Structures.jl) for details.

In [ ]:
# Geometry ============================================================== #
M   =   Geometry(
    xmin    =   0.0,                #   [ m ] 
    xmax    =   1000e3,             #   [ m ]
    ymin    =   -1000e3,            #   [ m ]
    ymax    =   0.0,                #   [ m ]
)
# ----------------------------------------------------------------------- #
# Referenceparameters =================================================== #
P   =   Physics(
    g       =   10.0,               #   Graviational Acceleration [m/s^2]
    ρ₀      =   4000.0,             #   Reference Density [kg/m^3]
    k       =   5.0,                #   Thermal Conductivity [ W/m/K ]
    cp      =   1250.0,             #   Heat capacity [ J/kg/K ]
    α       =   2.5e-5,             #   THermal Expansion [ K^-1 ]
    η₀      =   1e23,               #   Reference Viscosity [ Pa*s ]
    ΔT      =   1000.0,             #   Temperature Difference
)
# ----------------------------------------------------------------------- #

We can now define the **scaling constants**:

In [ ]:
# Define Scaling Constants ============================================== # 
S   =   ScalingConstants!(M,P)
# ----------------------------------------------------------------------- #

... and the **numerical grid**.

In [ ]:
# Numerical Grid ======================================================== #
NC  =   (
    x   =   100,
    y   =   100,
)
NV      =   (
    x   =   NC.x + 1,
    y   =   NC.y + 1,
)
Δ       =   GridSpacing(
    x   =   (M.xmax - M.xmin)/NC.x,
    y   =   (M.ymax - M.ymin)/NC.y,
)
# ----------------------------------------------------------------------- #

Next, we initialize the **fields for our variables**. Since we want to keep different *solvers* as available options, many fields need to be **allocated** here.

In [ ]:
# Data Arrays =========================================================== #
D       =   DataFields(
    Q       =   zeros(Float64,(NC...)),
    T       =   zeros(Float64,(NC...)),
    T0      =   zeros(Float64,(NC...)),
    T_ex    =   zeros(Float64,(NC.x+2,NC.y+2)),
    # T_exo   =   zeros(Float64,(NC.x+2,NC.y+2)),
    ρ       =   ones(Float64,(NC...)),
    # cp      =   ones(Float64,(NC...)),
    vx      =   zeros(Float64,(NV.x,NV.y+1)),
    vy      =   zeros(Float64,(NV.x+1,NV.y)),    
    Pt      =   zeros(Float64,(NC...)),
    vxc     =   zeros(Float64,(NC...)),
    vyc     =   zeros(Float64,(NC...)),
    vc      =   zeros(Float64,(NC...)),
    # wt      =   zeros(Float64,(NC...)),
    # wtv     =   zeros(Float64,(NV...)),
    ΔTtop   =   zeros(Float64,NC.x),
    ΔTbot   =   zeros(Float64,NC.x),
    # Tmax    =   0.0,
    # Tmin    =   0.0,
    # Tmean   =   0.0,
)
# # Heat Production Rate ------
# @. D.Q      =   P.Q₀
# ----------------------------------------------------------------------- #
# Needed for the defect correction solution ---
divV        =   zeros(Float64,NC...)
ε           =   (
    xx      =   zeros(Float64,NC...), 
    yy      =   zeros(Float64,NC...), 
    xy      =   zeros(Float64,NV...),
)
τ           =   (
    xx      =   zeros(Float64,NC...), 
    yy      =   zeros(Float64,NC...), 
    xy      =   zeros(Float64,NV...),
)
# Residuals ---
Fm     =    (
    x       =   zeros(Float64,NV.x, NC.y), 
    y       =   zeros(Float64,NC.x, NV.y)
)
FPt         =   zeros(Float64,NC...)
# ----------------------------------------------------------------------- #

Now we can determine the **temporal parameters**. In particular, we need to specify the **maximum time step** and the **maximum simulation time**.

For the maximum time step, we must ensure that the **stability criteria** are satisfied (these apply mainly when using certain finite difference methods).  
The stability criteria can be adjusted using scaling factors $\Delta facc$ and $\Delta facd$.

This means we must calculate the time step according to the **diffusion stability criterion** and the **Courant criterion**. The maximum time step is then set to the **minimum** of both values.

The maximum time and the maximum number of iterations are chosen **ad hoc** for now. In general, the simulation should stop once the **average velocity** no longer changes significantly, indicating that the system has reached a **steady-state**.

In [ ]:
# Time parameters ======================================================= #
T   =   TimeParameter(
    tmax    =   1000000.0,          #   [ Ma ]
    Δfacc   =   0.9,                #   Courant time factor
    Δfacd   =   0.9,                #   Diffusion time factor
    itmax   =   8000,               #   Maximum iterations
)
T.tmax      =   T.tmax*1e6*T.year   #   [ s ]
T.Δc        =   T.Δfacc * minimum((Δ.x,Δ.y)) / 
                    (sqrt(maximum(abs.(D.vx))^2 + maximum(abs.(D.vy))^2))
T.Δd        =   T.Δfacd * (1.0 / (2.0 * P.κ *(1.0/Δ.x^2 + 1/Δ.y^2)))

T.Δ         =   minimum([T.Δd,T.Δc])

Time        =   zeros(T.itmax)
Nus         =   zeros(T.itmax)
meanV       =   zeros(T.itmax)
meanT       =   zeros(T.itmax,NC.y+2)
find        =   0
# ----------------------------------------------------------------------- #

Now that we have defined all the **parameters and constants** required for scaling, we can apply the **scaling laws**.

In [ ]:
# Scaling laws ========================================================== #
ScaleParameters!(S,M,Δ,T,P,D)
# ----------------------------------------------------------------------- #

Mit Hilfe der skalierten Größen können wir nun auch die dimensionslosen Koordinaten für unser Modell definieren: 

In [ ]:
# Coordinates =========================================================== #
x       =   (
    c   =   LinRange(M.xmin+Δ.x/2,M.xmax-Δ.x/2,NC.x),
    ce  =   LinRange(M.xmin - Δ.x/2.0, M.xmax + Δ.x/2.0, NC.x+2),
    v   =   LinRange(M.xmin,M.xmax,NV.x),
)
y       =   (
    c   =   LinRange(M.ymin+Δ.y/2,M.ymax-Δ.y/2,NC.y),
    ce  =   LinRange(M.ymin - Δ.x/2.0, M.ymax + Δ.x/2.0, NC.y+2),
    v   =   LinRange(M.ymin,M.ymax,NV.y),
)
x1      =   (
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# ----------------------------------------------------------------------- #

Als Anfangsbedingungen nehmen wir ein mit der Tiefe, linear ansteigendes Temperaturprofil mit einer kleinen Gaussian Anomalie, wobei die Anfangsbedingung mit der Routine ```IniTemperature!()``` berechnet wird.

Bei den Randbedingungen müssen wir für die Temperatur- und die Impulsgleichung die Bedingungen festlegen. Dabei nehmen wir für die thermischen Randbedingungen an: 

1. Norden  -   Dirichlet
2. Süden   -   Dirichlet
3. Westen  -   Neumann
4. Osten   -   Neumann 

Für die Geschwindigkeitsrandbedingungen nehmen wir für alle Ränder *free slip* Bedingungen an. 

In [ ]:
# Initial Condition ===================================================== #
# Temperature ------
IniTemperature!(Ini.T,M,NC,D,x,y;Tb=P.Tbot,Ta=P.Ttop)
# ----------------------------------------------------------------------- #
# Boundary Conditions =================================================== #
# Temperature ------
TBC     = (
    type    = (W=:Neumann, E=:Neumann,N=:Dirichlet,S=:Dirichlet),
    val     = (W=zeros(NC.y),E=zeros(NC.y),
                    N=P.Ttop.*ones(NC.x),S=P.Tbot.*ones(NC.x)))
# Velocity ------
VBC     =   (
    type    =   (E=:freeslip,W=:freeslip,S=:freeslip,N=:freeslip),
    val     =   (E=zeros(NV.y),W=zeros(NV.y),S=zeros(NV.x),N=zeros(NV.x)),
)
# ----------------------------------------------------------------------- #

Nun wird die *Rayleigh* Zahl berechnet:

>Beachte: Da wir die Namen der Dateien von der Rayleigh Zahl abhängig machen, müssen wir nun hier auch die Namen definieren. 

In [ ]:
# Rayleigh Number ======================================================= #
# if P.Ra < 0
    P.Ra     =   P.ρ₀*P.g*P.α*P.ΔT*S.hsc^3/P.η₀/P.κ
# else
#     P.η₀     =   P.ρ₀*P.g*P.α*P.ΔT*S.hsc^3/P.Ra/P.κ
# end
filename    =   string("13_Blankenbach_",@sprintf("%.2e",P.Ra),
                        "_",NC.x,"_",NC.y,
                        "_",Ini.T)
# ----------------------------------------------------------------------- #

Im folgenden werden die Parameter für die linearen Gleichungssysteme bestimmt:

>Beachte: Für den Benchmark wählen wir das Crank-Nicolson Verfahren für die Wärmeleitgleichung, die Semi-Lagrange Methode für die Advektion der Temperatur, und die Defekt Korretur Iteration für die Impulserhalung. Gerne dürfen auch andere Solver verwendet werden.

In [ ]:
# Linear System of Equations ============================================ #
# Momentum Conservation Equation (MCE) ------
niter  =   50
ϵ      =   1e-10
off    = [  NV.x*NC.y,                          # vx
            NV.x*NC.y + NC.x*NV.y,              # vy
            NV.x*NC.y + NC.x*NV.y + NC.x*NC.y ] # Pt
Num    =    (
    Vx  =   reshape(1:NV.x*NC.y, NV.x, NC.y), 
    Vy  =   reshape(off[1]+1:off[1]+NC.x*NV.y, NC.x, NV.y), 
    Pt  =   reshape(off[2]+1:off[2]+NC.x*NC.y,NC...),
    T   =   reshape(1:NC.x*NC.y, NC.x, NC.y),
)
ndof    =   maximum(Num.T)        
# Energy Conservation Equation (ECE) ------
K1      =   ExtendableSparseMatrix(ndof,ndof)
K2      =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(ndof)
# ----------------------------------------------------------------------- #

Nun können wir die Gleichungen in der Zeitschleife lösen. Dabei müssen die folgenden Schritte berücksichtig werden: 

1. Initialisierung des unbekannten Vektors und der rechten Seite für die Impulserhaltung
2. Berechnung der Zeit
3. Null Setzung der Geschwindigkeits- und Druckfelder
4. Lösen der Impulserhaltung
5. Update der Geschwindigkeits- und Druckfelder
6. Bestimmung der Geschwindikeiten auf den *Centroids* 
7. Anpassung der maximalen Zeitschrittlänge
8. Darstellung der Geschwindigkeit, Temperatur, und Dichte
9. Lösen der Advektionsgleichung
10. Lösen der Diffusionsgleichung
11. Berechnung der statistischen Analysewerte (*Nusselt* Zahl, $V_{RMS}$, etc.)

In [ ]:
# Zeitschleife =========================================================== #
for it = 1:T.itmax
    χ       =   zeros(maximum(Num.Pt))      #   Unbekannter Vektor IEG
    rhsM    =   zeros(maximum(Num.Pt))      #   Rechte Seite IEG
    if it>1
        Time[it]  =   Time[it-1] + T.Δ
    end
    @printf("Time step: #%04d, Time [non-dim]: %04e\n ",it,
                    Time[it])
    # IEG ------
    D.vx    .=  0.0
    D.vy    .=  0.0 
    D.Pt    .=  0.0
    # Residual Calculation ------
    @. D.ρ  =   -P.Ra*D.T
    for iter = 1:niter
        Residuals2Dc!(D,VBC,ε,τ,divV,Δ,1.0,1.0,Fm,FPt)
        rhsM[Num.Vx]    =   Fm.x[:]
        rhsM[Num.Vy]    =   Fm.y[:]
        rhsM[Num.Pt]    =   FPt[:]
        @printf("||R_M|| = %1.4e\n", norm(rhsM)/length(rhsM))
            norm(rhsM)/length(rhsM) < ϵ ? break : nothing
        # Update K ------
        K       =   Assemblyc(NC, NV, Δ, 1.0, VBC, Num)
        # Solving Linear System of Equations ------
        χ      =   - K \ rhsM
        # Update unknown variables ------
        D.vx[:,2:end-1]     .+=  χ[Num.Vx]
        D.vy[2:end-1,:]     .+=  χ[Num.Vy]
        D.Pt                .+=  χ[Num.Pt]
    end
    D.ρ  =   ones(NC...)
    # ======
    # Calculate Velocity on the Centroids ------
    for i = 1:NC.x
        for j = 1:NC.y
            D.vxc[i,j]  = (D.vx[i,j+1] + D.vx[i+1,j+1])/2
            D.vyc[i,j]  = (D.vy[i+1,j] + D.vy[i+1,j+1])/2
        end
    end
    @. D.vc        = sqrt(D.vxc^2 + D.vyc^2)
    # ---
    @show(maximum(D.vc))
    @show(minimum(D.Pt))
    @show(maximum(D.Pt))
    # Calculate time stepping =========================================== #
    T.Δc        =   T.Δfacc * minimum((Δ.x,Δ.y)) / 
            (sqrt(maximum(abs.(D.vx))^2 + maximum(abs.(D.vy))^2))
    T.Δd        =   T.Δfacd * (1.0 / (2.0 *(1.0/Δ.x^2 + 1/Δ.y^2)))
    T.Δ         =   minimum([T.Δd,T.Δc])
    if Time[it] > T.tmax
        T.Δ         =   T.tmax - Time[it-1]
        Time[it]    =   Time[it-1] + T.Δ
        it          =   T.itmax
    end
    # Plot ============================================================== #
    if mod(it,10) == 0 || it == T.itmax || it == 1
        p = heatmap(x.c,y.c,D.T',
                xlabel="x",ylabel="y",colorbar=true,
                title="Temperature",color=cgrad(:lajolla),
                aspect_ratio=:equal,xlims=(M.xmin, M.xmax),
                ylims=(M.ymin, M.ymax))
        contour!(p,x.c,y.c,D.T',lw=1,
                    color="white",cbar=false,alpha=0.5)
        quiver!(p,x.c2d[1:Pl.qinc:end,1:Pl.qinc:end],
            y.c2d[1:Pl.qinc:end,1:Pl.qinc:end],
            quiver=(D.vxc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc,
                    D.vyc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc),
            la=0.5,color="black",alpha=0.5)
        if save_fig == 1
            Plots.frame(anim)
        elseif save_fig == 0
            display(p)
        end
    end
    # ------------------------------------------------------------------- #
    # Advection ========================================================= #
    semilagc2D!(D.T,D.T_ex,D.vxc,D.vyc,[],[],x,y,T.Δ)
    # ------------------------------------------------------------------- #
    # Diffusion ========================================================= #
    CNA2Dc!(D, 1.0, Δ.x, Δ.y, T.Δ, D.ρ, 1.0, NC, TBC, rhs, K1, K2, Num)
    # ------------------------------------------------------------------- #
    # Nusselt Number ==================================================== #
    # Grid structure at the surface ---
    #   o - Centroids
    #   x - Vertices 
    #   □ - Ghost Nodes
    #
    #   □          □           □            □
    #   
    #        x --------- x --------- x
    #        |           |           |
    #   □    |     o     |     o     |      □ 
    #        |           |           |
    #        x --------- x --------- x      
    #        |           |           |
    #   □    |     o     |     o     |      □
    # --- 
    # Get temperature at the vertices 
    Tv1     =   zeros(NV.x,1)
    Tv2     =   zeros(NV.x,1)
    @. Tv1  =   (D.T_ex[1:end-1,end] + D.T_ex[2:end,end] + 
                    D.T_ex[1:end-1,end-1] + D.T_ex[2:end,end-1])/4
    @. Tv2  =   (D.T_ex[1:end-1,end-1] + D.T_ex[2:end,end-1] + 
                    D.T_ex[1:end-1,end-2] + D.T_ex[2:end,end-2])/4
    # Calculate temperature gradient --- 
    dTdy    =   zeros(NV.x,1)
    @. dTdy =   -(Tv1 - Tv2)/Δ.y
    # Calculate Nusselt number ---
    # Midpoint integration -
    # for i=1:NC.x
    #     Nus[it] +=   ((dTdy[i]+dTdy[i+1])/2)*Δ.x
    # end
    # Trapezoidal integration -
    for i = 1:NV.x
        if i == 1 || i == NV.x
            afac = 1
        else
            afac = 2
        end
        Nus[it]     += afac * dTdy[i]
    end
    Nus[it]     *=   Δ.x/2
    # Mean Temperature ---
    meanT[it,:] =   mean(D.T_ex,dims=1)
    # Root Mean Square Velocity ---
    meanV[it]   =   mean(D.vc)
    # ------------------------------------------------------------------- #
    # Check break ======================================================= #
    # If the maximum time is reached or if the models reaches steady 
    # state the time loop is stoped! 
    if Time[it] > 0.0038
        epsC    =   1e-3; 
        ind     =   findfirst(Time .> 
                        (Time[it] - 0.0038))
        epsV    =   std(meanV[ind:it])
        if save_fig == 1
            plot!(k,(it,log10((epsV))),
                xlabel="it",ylabel="log₁₀(εᵥ)",label="",
                markershape=:circle,markercolor=:black)
        end
        find    =   it
        @printf("ε_V = %g, ε_C = %g \n",epsV,epsC)
        if Time[it] >= T.tmax
            @printf("Maximum time reached!\n")
            find    =   it
            break
        elseif (epsV <= epsC)
            @printf("Convection reaches steady state!\n")
            find    =   it
            break
        end
    end
    # --------------------------------------------------------------- #
    @printf("\n")
end

Im Folgenden werden die jeweiligen Abbildungen gespeichert. 

In [ ]:
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
    foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
end
# Save final figure ===================================================== #
p2 = heatmap(x.c,y.c,D.T',
            xlabel="x",ylabel="y",colorbar=true,
            title="Temperature",color=cgrad(:lajolla),
            aspect_ratio=:equal,xlims=(M.xmin, M.xmax),
            ylims=(M.ymin, M.ymax))
    contour!(p2,x.c,y.c,D.T',lw=1,
                    color="white",cbar=false,
                    alpha=0.5)
    quiver!(p2,x.c2d[1:Pl.qinc:end,1:Pl.qinc:end],
            y.c2d[1:Pl.qinc:end,1:Pl.qinc:end],
            quiver=(D.vxc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc,
                    D.vyc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc),
            la=0.5,color="black",
            alpha=0.5)
if save_fig == 1
    savefig(k,string("./Results/13_BlankenbachBenchmark_iterations_",@sprintf("%.2e",P.Ra),
            "_",NC.x,"_",NC.y,
            "_",Ini.T,".png"))
    savefig(p2,string("./Results/13_BlankenbachBenchmark_Final_Stage_",@sprintf("%.2e",P.Ra),
            "_",NC.x,"_",NC.y,"_it_",find,"_",
            Ini.T,".png"))
elseif save_fig == 0
    display(p2)
end
# ----------------------------------------------------------------------- #

Zum Schluss werden die statistischen Analyseparameter noch dargestellt und gespeichert. 

In [ ]:
# Plot time serieses ==================================================== #
q2  =   plot(Time[1:find],Nus[1:find],
            xlabel="Time [ non-dim ]", ylabel="Nus",label="",
            layout=(2,1),suplot=1)
plot!(q2,Time[1:find],B.Nu[B.Nr].*ones(find,1),
            lw=0.5,color="red",linestyle=:dash,alpha=0.5,label="",
            layout=(2,1),suplot=1)
plot!(q2,Time[1:find],meanV[1:find],
            xlabel="Time [ non-dim ]", ylabel="V_{RMS}",label="",
            layout=(2,1),subplot=2)
plot!(q2,Time[1:find],B.Vrms[B.Nr].*ones(find,1),
            lw=0.5,color="red",linestyle=:dash,alpha=0.5,label="",
            layout=(2,1),subplot=2)
if save_fig == 1
    savefig(q2,string("./Results/13_BlankenbachBenchmark_TimeSeries_",@sprintf("%.2e",P.Ra),
                        "_",NC.x,"_",NC.y,"_",Ini.T,".png"))
elseif save_fig == 0
    display(q2)
end
# ----------------------------------------------------------------------- #
# Vertical temperature profiles ========================================= #
ind1    =   Int64((M.xmax-M.xmin)/2/Δ.x+1)
ind2    =   Int64((M.xmax-M.xmin)/2/Δ.x)
Tprof   =   (D.T[ind1,:]+D.T[ind2,:])/2
q3  =   plot(Tprof,y.c,
        xlabel="T",ylabel="y",label=false)
scatter!(q3,(B.Tmin[B.Nr],-(1-B.ymin[B.Nr])),
            markershape=:rect,markersize=3,
            markercolor=:black,label=false)
scatter!(q3,(B.Tmax[B.Nr],-(1-B.ymax[B.Nr])),
            markershape=:rect,markersize=3,
            markercolor=:black,label=false)
if save_fig == 1
    savefig(q3,string("./Results/13_BlankenbachBenchmark_Profiles_",@sprintf("%.2e",P.Ra),
                        "_",NC.x,"_",NC.y,"_",Ini.T,".png"))
elseif save_fig == 0
    display(q3)
end
# ----------------------------------------------------------------------- #
stop=time()
println(stop-start)

In [ ]:
@show find

In [ ]:
@show stop-start